In [1]:
import sys
sys.path.append('..')
import numpy as np, pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from skopt.space import Real, Integer, Categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from nestedcvtraining.api import find_best_model
from nestedcvtraining.switch_case import SwitchCase
from nestedcvtraining.under_sampling_classifier import UnderSamplingClassifier
from skopt import gbrt_minimize
from sklearn.metrics import log_loss

In [2]:
from sklearn.datasets import make_classification
X, y = make_classification(
    n_samples=50,
    n_features=20, 
    n_redundant=5, 
    n_informative=5, 
    n_classes=2, 
    n_clusters_per_class=3,
    flip_y=0.05,
    class_sep=0.7,
    weights=[0.8, 0.2],
    random_state=42
)

In [3]:
from collections import Counter
Counter(y)

Counter({0: 36, 1: 14})

In [4]:
resampler = SwitchCase(
    cases=[
        (
            "resampler_1",
            SMOTE(k_neighbors=3)
        ),
        (
            "resampler_2",
            "passthrough"
        )
    ],
    switch="resampler_1"
)

preprocessor = SwitchCase(
    cases=[
        (
            "prep_1",
            SkPipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", PCA(n_components=5))
            ])
        ),
        (
            "prep_2",
            SkPipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", SelectKBest(mutual_info_classif, k=5)),
            ])
        ),
        (
            "prep_3",
            "passthrough"
        )
    ],
    switch="prep_1"
)

model = SwitchCase(
    cases=[
        (
            "model_1",
            LogisticRegression()
        ),
        (
            "model_2",
            RandomForestClassifier()
        )
    ],
    switch="model_1"
)

clf = ImbPipeline(
    [("resampler", resampler), ("preprocessor", preprocessor), ("model", model)]
)

search_space= [
    Categorical(["resampler_1", "resampler_2"], name="resampler__switch"),
    Categorical(["prep_1", "prep_2", "prep_3"], name="preprocessor__switch"),
    Categorical(["model_1", "model_2"], name="model__switch"),
    Categorical(["minority", "all"], name="resampler__resampler_1__sampling_strategy"),
    Integer(5, 15, name="model__model_2__max_depth")
]


In [ ]:
best_model, best_params, report = find_best_model(
    X=X,
    y=y,
    model=clf,
    search_space=search_space,
    verbose=False,
    k_inner=49,
    k_outer=50,
    n_initial_points=10,
    n_calls=10,
    calibrate="only_best",
    calibrate_params={"method": "isotonic"},
    optimizing_metric=make_scorer(log_loss, greater_is_better=False, needs_proba=True, labels=[0, 1]),
    other_metrics={"acc": "accuracy"},
    skopt_func=gbrt_minimize
)

Looping over outer folds
Looping over outer folds
Looping over outer folds


In [15]:
neg_log_loss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

In [17]:
log_loss(["spam", "ham", "ham", "spam"],
         [[.1, .9], [.9, .1], [.8, .2], [.35, .65]])

0.21616187468057912

In [16]:
neg_log_loss(["spam", "ham", "ham", "spam"],
         [[.1, .9], [.9, .1], [.8, .2], [.35, .65]])

TypeError: __call__() missing 1 required positional argument: 'y_true'

In [14]:
neg_log_loss

make_scorer(log_loss)

In [11]:
_, y_idx, y_inv = np.unique(y, return_index=True, return_inverse=True)
# y_inv encodes y according to lexicographic order. We invert y_idx to
# map the classes so that they are encoded by order of appearance:
# 0 represents the first label appearing in y, 1 the second, etc.
_, class_perm = np.unique(y_idx, return_inverse=True)
y_encoded = class_perm[y_inv]

n_classes = len(y_idx)
y_counts = np.bincount(y_encoded)
min_groups = np.min(y_counts)

In [12]:
y_counts

array([36, 14])

In [14]:
np.all(50 > y_counts)

True

In [15]:
np.bincount(y)

array([36, 14])